In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import dataloaders
from dataloaders.dataloader import loadDataset
import micronas
from micronas import MicroNas, MicroNasMCU
BASE_DATASET_FOLDER = r"/Users/king/Github/timeseries-datasets/datasets/"

In [5]:
# Get the data
# train_data, vali_data, test_data = loadDataset(BASE_DATASET_FOLDER, "ucihar")

In [6]:
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset
import pickle
import torch
import os
from torch.utils.data import DataLoader
from micronas.config import Config

class InMemoryDataset(Dataset):
    def __init__(self, data_x, data_y):
        assert len(data_x) == len(data_y)
        self.data_x = data_x
        self.data_y = data_y

    def __len__(self):
        return len(self.data_x)
    
    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]
    
    def save(self, name):
        if not os.path.exists(os.path.join(BASE_DATASET_FOLDER, "pickle")):
            os.makedirs(os.path.join(BASE_DATASET_FOLDER, "pickle"))

        with open(os.path.join(BASE_DATASET_FOLDER, "pickle", name), "wb") as f:
            pickle.dump({"data_x": self.data_x, "data_y": self.data_y}, f)

    @staticmethod
    def load(name):
        with open(os.path.join(BASE_DATASET_FOLDER, "pickle", name), "rb") as f:
            data = pickle.load(f)
        data_x = torch.tensor(data["data_x"], dtype=torch.float32)
        data_y = torch.tensor(data["data_y"], dtype=torch.long)
        return InMemoryDataset(data_x, data_y)

def extract_data(data_set):
    train_data_x = []
    train_data_y = []
    for x in tqdm(data_set):
        train_data_x.append(x[0])
        train_data_y.append(x[1])
    train_data_x = np.array(train_data_x)
    train_data_y = np.array(train_data_y)
    return train_data_x, train_data_y


# vali_data_x, vali_data_y = extract_data(vali_data)
# vali_dataset = InMemoryDataset(vali_data_x, vali_data_y)
# vali_dataset.save("ucihar_vali.pickle")

# train_data_x, train_data_y = extract_data(train_data)
# train_dataset = InMemoryDataset(train_data_x, train_data_y)
# train_dataset.save("ucihar_train.pickle")

vali_dataset = InMemoryDataset.load("ucihar_vali.pickle")
train_dataset = InMemoryDataset.load("ucihar_train.pickle")


In [7]:
print(next(iter(vali_dataset))[0].device)

cpu


In [8]:
# mNas = MicroNas(train_dataset, vali_dataset, vali_dataset, 6)
# mNas.search(MicroNasMCU.NUCLEOF446RE, latency_limit=None, memory_limit=30000, search_epochs=1, compute_unit="cpu")


In [14]:
from micronas import MicroNas, MicroNasMCU
from micronas.Nas.Networks.Pytorch.SearchNet import SearchNet
from micronas.Nas.SearchStrategy.DnasStrategy import DNasStrategy
from micronas.Nas.SearchStrategy.RandomSearchStrategy import RandomSearchStrategy

model = MicroNas(train_dataset, vali_dataset, vali_dataset, 6)

input_dims = next(iter(train_dataset))[0].shape
print(input_dims)
search_space = SearchNet(input_dims=input_dims, num_classes=6)
search_strategy = DNasStrategy(search_space)
search_strategy = RandomSearchStrategy(search_space)

model.compile(search_space, search_strategy)

model.fit(MicroNasMCU.NUCLEOF446RE, latency_limit=None, memory_limit=30000, search_epochs=1, compute_unit="cpu")

torch.Size([128, 9])
#Time_reduce:  3
#Sensor_fusion:  1
not included:  Dyn_Conv2D
not included:  GlobalAveragePooling
not included:  LogSoftMax
Paralellv2_weights_keras:  None


TypeError: empty_like(): argument 'input' (position 1) must be Tensor, not NoneType